In [1]:
def INCLUDE(target, ipynb, key):
    import json
    with open(ipynb) as file:
        data = json.load(file)
        for cell in data['cells']:
            lines = cell['source']
            if type(lines) == str:
                lines = lines.split('\n')
            line0 = (lines or [''])[0].strip()
            if cell['cell_type'] == 'code' and line0 == f'### {key} ###':
                code = '\n'.join(lines)
                exec(code, target)

ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks
ttt-3.ipynb  ttt-tactics  ttt-tactics-tf.ipynb	ttt-tactics-train.ipynb


In [2]:
notebook = f'./ttt-tactics-tf.ipynb'

INCLUDE(globals(), notebook, 'HEAD')
TESTS = False
INCLUDE(globals(), notebook, 'GAME')
INCLUDE(globals(), notebook, 'DATA')
INCLUDE(globals(), notebook, 'MODEL')

ipynb = 'ttt-tactics-train'


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8748468073563546901
,
 name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 871901071075597344
physical_device_desc: "device: XLA_CPU device"
]
tensorflow: 2.3.0
DIR = ./ttt-tactics
<class '__main__.AlphaZeroModel'>


In [ ]:
model   = AlphaZeroModel()
minloss = 3.0
iters   = 10_000
while iters > 0:
    iters -= 1

    models = sortedfiles(f'{DIR}/model-*.h5')
    model.load(models[-1])
    num = int(re.findall(r'\d+', str(model))[0]) 
    print('num =', num)

    path    = f'{DIR}/data-*.tfrec'
    files   = sortedfiles(path)
    dataset = loadsamples(files, buffer_size=2048, batch_size=64, seed=None)
    print(f'found {len(files)} files at {path}')

    schedule = { 0:0.0001 }
    def scheduler(epoch, lr): 
        if epoch in schedule: return schedule[epoch]  
        else:                 return lr

    history = model.fit(
        x                   = dataset,
        epochs              = 10,
        verbose             = 1,
        callbacks           = [LearningRateScheduler(scheduler)],
        workers             = 10,
        use_multiprocessing = False)
    history = history.history

    loss = history['loss'][-1]
    ok   = loss < minloss
    if ok:
        minloss = loss
        model.save(f'{DIR}/model-{(num+1):0>3d}.h5')

    plt.figure()
    plt.plot(history['loss'],        label='total loss')
    plt.plot(history['policy_loss'], label='policy loss')
    plt.plot(history['value_loss'],  label='value loss')
    plt.title('training history')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc="upper left")
    plt.savefig(f'{DIR}/{str(model)}.png')
    plt.show()

    ok = 'OK' if ok else 'NO'
    NOTIFY(f'{str(model)} {ok}, minloss: {minloss}, loss: {loss}, shell={str(ipython)}', title=ipynb)


./ttt-tactics/model-000.h5 Oct 06 2020 14:41:29
num = 0
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Do not call `graph_parents`.
found 12 files at ./ttt-tactics/data-*.tfrec
Epoch 1/10
6068/6068 [==============================] - 393s 65ms/step - loss: 3.1917 - value_loss: 0.7800 - policy_loss: 2.4117 - value_lr: 9.9998e-05
Epoch 2/10
2229/6068 [==========>...................] - ETA: 4:07 - loss: 2.6670 - value_loss: 0.7683 - policy_loss: 1.8987 - value_lr: 1.0000e-04